<a href="https://colab.research.google.com/github/0xVolt/whats-up-doc/blob/main/test/notebooks/model-blending/blend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Merging CodeLLMs to Create an Efficant Low-Memory Quantized Model for `whats-up-doc`

## Download and Install `mergekit`

In [1]:
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .

fatal: destination path 'mergekit' already exists and is not an empty directory.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mergekit (pyproject.toml) ... done


## Create the YAML Config File to Merge Models with SLERP

In [2]:
import os
import yaml
from transformers import AutoModelWithLMHead, AutoTokenizer, pipeline

In [3]:
!pip install huggingface-cli

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

### Write Config Script

In [5]:
# Set the resultant model's name
MODEL_NAME = 'whats-up-llamas'

MODEL_1 = "codellama/CodeLlama-7b-Instruct-hf"
MODEL_2 = "meta-llama/Meta-Llama-3-8B-Instruct"

OUTPUT_DIR = "merged_model"

# LAYERS_MODEL_1 = list(range(0, 32))  # Layer range for MODEL_1
# LAYERS_MODEL_2 = list(range(0, 24))  # Layer range for MODEL_2

#### SLERP

In [24]:
yamlConfigSLERPLlamas = f"""
slices:
  - sources:
      - model: {MODEL_1}
        layer_range: [0, 32]
      - model: {MODEL_2}
        layer_range: [0, 32]
merge_method: slerp
base_model: {MODEL_1}
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - filter: embed_tokens.weight
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: input_layernorm.weight
      value: [0, 0.5, 0.3, 0.7, 1]
  slerp_coeff: 0.5
dtype: bfloat16
"""

#### Passthrough

In [15]:
yamlConfigPassthrough = """
slices:
  - sources:
    - model: OpenPipe/mistral-ft-optimized-1218
      layer_range: [0, 32]
  - sources:
    - model: mlabonne/NeuralHermes-2.5-Mistral-7B
      layer_range: [24, 32]
merge_method: passthrough
dtype: bfloat16

"""

*Note: If you were to do this locally, instead of putting in the models' card name under `model`, you would specify the path to the model you downloaded from huggingface.*

### Save Config Script

In [22]:
# Save the YAML configuration to a file
yamlFileName = "config.yaml"
with open(yamlFileName, "w") as f:
    f.write(yamlConfigSLERPLlamas)

## Merge Models

In [23]:
cmd = f"mergekit-yaml {yamlFileName} {OUTPUT_DIR} --allow-crimes --copy-tokenizer --out-shard-size 1B --low-cpu-memory --write-model-card --lazy-unpickle"
!{cmd}

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Warmup loader cache:   0% 0/2 [00:00<?, ?it/s]
Fetching 10 files: 100% 10/10 [00:00<00:00, 34407.74it/s]
Warmup loader cache:  50% 1/2 [00:00<00:00,  2.90it/s]
Fetching 11 files: 100% 11/11 [00:00<00:00, 99220.09it/s]
Warmup loader cache: 100% 2/2 [00:00<00:00,  3.85it/s]
Traceback (most recent call last):
  File "/usr/local/bin/mergekit-yaml", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/content/mergekit/mergekit/options.py", line 82, in

## Load the Blended Model

In [10]:
# Step 6: Load the merged model and tokenizer
loadModel = f"{OUTPUT_DIR}/{MODEL_NAME}"

tokenizer = AutoTokenizer.from_pretrained(loadModel)
model = AutoModelWithLMHead.from_pretrained(loadModel)

# Step 7: Run inference on the merged model
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
input_text = "def calculate_sum(num1, num2):\n    # Code to be generated"
generated_text = generator(input_text, max_length=50, num_return_sequences=1)[0]["generated_text"]
print("Generated Code:", generated_text)

OSError: merged_model/whats-up-llamas is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`